# Businesses - Brand and Competition Locations

The geography level methods and enrichment methods detailed in the previous notebooks enable retrieving location and demographic information describing where and who customers are. Next though, we need to retrieve information about the retail landscape surrounding these potential customers. Knowing the locations of brand and competitor locations is essential to modeling the effects of competition and cannibalization.

In [1]:
from pathlib import Path
import sys

project_parent = Path('./').absolute().parent
dir_data = project_parent/'data'
dir_int = dir_data/'interim'
gdb_int = dir_int/'interim.gdb'

# import the project package from the project package path
# ideally will be imported using 'from arcgis import dm'
sys.path.append(str(project_parent/'src'))
import dm

# load the "autoreload" extension so as src code is changed, the changes are picked up in the dataframe
%load_ext autoreload
%autoreload 2

D:\projects\demographic-modeling-module\src\dm\_modify_geoaccessor.py:31: UserWarning: registration of accessor <class 'dm._modify_geoaccessor.GeoAccessorIO'> under name 'spatial' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexistingattribute with the same name.
  class GeoAccessorIO(GeoAccessor):


Getting started is not dramactically different than before - instantiate a country object intsance and delineate an area of interest.

In [2]:
usa = dm.Country('USA')
aoi_df = usa.cbsas.get('seattle')
aoi_df.spatial.plot()

MapView(layout=Layout(height='400px', width='100%'))

## Brand Locations

Then, business locations can be retrieved using the business name.

In [3]:
biz_df = usa.business.get_by_name('Ace Hardware', aoi_df)
biz_df.head()

,OBJECTID,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,...,EMPNUM,FRNCOD,ISCODE,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REC_TYPE,SHAPE
0,1742102,371889957,GRAHAM ACE HARDWARE,224TH ST E,GRAHAM,WA,Washington,98338,5704,44413005,...,15.0,1,,5,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -13614273.536475636, ""y"": 5950909.044418..."
1,5853272,460556608,OAKBROOK ACE HARDWARE,STEILACOOM BLVD SW,LAKEWOOD,WA,Washington,98498,6154,44413005,...,9.0,1,,3,StreetAddress,M,100.0,INFOGROUP,0,"{""x"": -13641713.790956184, ""y"": 5971504.304607..."
2,2830418,405129289,GIG HARBOR ACE HARDWARE,POINT FOSDICK DR NW,GIG HARBOR,WA,Washington,98335,1711,44413005,...,38.0,1,,7,StreetAddress,M,100.0,INFOGROUP,0,"{""x"": -13645465.257795917, ""y"": 5991409.736829..."
3,2690144,404324160,AGRISHOP ACE HARDWARE,S 12TH ST,TACOMA,WA,Washington,98405,3004,44413005,...,10.0,1,,5,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -13632607.856609292, ""y"": 5982959.933531..."
4,2215877,401684669,ACE HARDWARE & PAINT CTR,N PEARL ST,TACOMA,WA,Washington,98407,2416,44413005,...,23.0,1,,8,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -13638162.699199878, ""y"": 5986847.462380..."


In [4]:
biz_df.spatial.plot()

MapView(layout=Layout(height='400px', width='100%'))

Businesses can also be looked up by their category code. By default, this uses NAICS, but SIC can also be used.

In [5]:
naics_df = usa.business.get_by_code(44413005, aoi_df)

print(f'rows retrieved: {len(naics_df.index)}\n')

naics_df.head()

rows retrieved: 128



,OBJECTID,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,...,EMPNUM,FRNCOD,ISCODE,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REC_TYPE,SHAPE
0,1742102,371889957,GRAHAM ACE HARDWARE,224TH ST E,GRAHAM,WA,Washington,98338,5704,44413005,...,15.0,1,,5,PointAddress,M,100.0000,INFOGROUP,0,"{""x"": -13614273.536475636, ""y"": 5950909.044418..."
1,11948738,740405810,CASCADE WHOLESALE HARDWARE,PACIFIC HWY SW,LAKEWOOD,WA,Washington,98499,1027,44413005,...,6.0,,,3,PointAddress,M,100.0000,INFOGROUP,0,"{""x"": -13638463.261825016, ""y"": 5965397.528925..."
2,791119,175096239,LAKEWOOD HARDWARE & PAINT,LAKEWOOD DR SW,LAKEWOOD,WA,Washington,98499,3966,44413005,...,15.0,B,,5,PointAddress,M,100.0000,INFOGROUP,0,"{""x"": -13637583.837847747, ""y"": 5970734.581986..."
3,5853272,460556608,OAKBROOK ACE HARDWARE,STEILACOOM BLVD SW,LAKEWOOD,WA,Washington,98498,6154,44413005,...,9.0,1,,3,StreetAddress,M,100.0000,INFOGROUP,0,"{""x"": -13641713.790956184, ""y"": 5971504.304607..."
4,9170068,714960080,OAKBROOK ACE,ISTEILACOOM BLVD SW,LAKEWOOD,WA,Washington,98498,,44413005,...,6.0,,,3,StreetAddress,M,90.9063,INFOGROUP,0,"{""x"": -13641713.790956184, ""y"": 5971504.304607..."


Further, it frequently is also useful to retrieve broader categories by simply truncating two digits at a time to be less specific.

In [6]:
naics_big_df = usa.business.get_by_code(4441, aoi_df)

print(f'rows retrieved: {len(naics_big_df.index):,}\n')

naics_big_df.head()

rows retrieved: 1,664



,OBJECTID,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,...,EMPNUM,FRNCOD,ISCODE,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REC_TYPE,SHAPE
0,964999,218001303,TIN MEN SUPPLY INC,STATE ROUTE 507 S,ROY,WA,Washington,98580,,44419055,...,1.0,,,3,StreetAddress,T,100.0,INFOGROUP,0,"{""x"": -13642526.423238974, ""y"": 5932693.654005..."
1,6036182,481554129,MILES SAND & GRAVEL CO,STATE ROUTE 507 S,ROY,WA,Washington,98580,8745,44419012,...,15.0,,,5,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -13642370.575951867, ""y"": 5938500.167339..."
2,12885902,961135167,MC KENNA DOOR,46TH AVE S,ROY,WA,Washington,98580,7623,44419017,...,2.0,,,2,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -13635724.802351505, ""y"": 5932171.909401..."
3,12707346,904945607,GENERATION GLASS CO INC,8TH AVE S,ROY,WA,Washington,98580,0018,44419031,...,6.0,,,2,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -13630392.598742511, ""y"": 5925489.813913..."
4,12978535,973622731,ARROW LUMBER & HARDWARE,CENTER ST E,EATONVILLE,WA,Washington,98328,7449,44419044,...,10.0,,,5,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -13610110.18751997, ""y"": 5920245.5056171..."


## Competitor Locations

Also, competitors can easily be located using the previously retrieved data, or using a brand name to exclude from retrieved results. The latter is extremely useful when using locations the organization already has for the brand locations, but needing competitor locations for modeling.

In [7]:
# comp_df = usa.business.get_competition(biz_df, aoi_df)
comp_df = usa.business.get_competition('Ace Hardware', aoi_df)

comp_df.head()

,OBJECTID,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,...,EMPNUM,FRNCOD,ISCODE,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REC_TYPE,SHAPE
0,11948738,740405810,CASCADE WHOLESALE HARDWARE,PACIFIC HWY SW,LAKEWOOD,WA,Washington,98499,1027,44413005,...,6.0,,,3,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -13638463.261825016, ""y"": 5965397.528925..."
1,791119,175096239,LAKEWOOD HARDWARE & PAINT,LAKEWOOD DR SW,LAKEWOOD,WA,Washington,98499,3966,44413005,...,15.0,B,,5,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -13637583.837847747, ""y"": 5970734.581986..."
2,9459298,717157403,SWINGS & THINGS,BRIDGEPORT WAY SW,LAKEWOOD,WA,Washington,98499,2645,44413005,...,1.0,,,1,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -13638585.713264894, ""y"": 5970980.230739..."
3,9417066,716987694,WILCO FARM STORES-GIG HARBOR,E HIGHWAY 20,GIG HARBOR,WA,Washington,98335,,44413005,...,6.0,,,3,Postal,M,100.0,INFOGROUP,0,"{""x"": -13645476.389744991, ""y"": 5992394.715097..."
4,4190467,425066759,MCLENDON HARDWARE INC,N PEARL ST,TACOMA,WA,Washington,98406,7921,44413005,...,38.0,,,7,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -13638251.754792504, ""y"": 5984485.212081..."


Also, just as before, the returned results are Spatially Enabled DataFrames, so they can be easily saved for follow on analysis or visualization in ArcGIS Pro.

In [8]:
biz_df.spatial.to_featureclass(gdb_int/'loc_brand')

'D:\\projects\\demographic-modeling-module\\data\\interim\\interim.gdb\\loc_brand'

In [9]:
comp_df.spatial.to_featureclass(gdb_int/'loc_comp')

'D:\\projects\\demographic-modeling-module\\data\\interim\\interim.gdb\\loc_comp'